In [ ]:
from collections import OrderedDict
from pathlib import Path

import mne
from mne_bids import (BIDSPath, read_raw_bids, write_raw_bids)
import numpy as np

import IO

In [ ]:
# Enter filename and BIDS root
bids_root = Path('/Users/richardkoehler/Documents/Neurology_Data/BIDS Berlin/')
filename = 'sub-002_ses-20200131_task-selfpacedrotation202001310001_run-4_ieeg'

subject, session, task, run = IO.get_subject_sess_task_run(filename)
dataype = 'ieeg'
bids_path = BIDSPath(subject=subject, session=session, task=task, run=run, datatype=dataype, root=bids_root)

In [ ]:
# If preload is set to TRUE, write_raw_bids might not work. Only load_data if necessary.
raw = read_raw_bids(bids_path=bids_path, extra_params=dict(preload=False), verbose=False)

In [ ]:
# Read in file containing electrode locations
electr_file = bids_path.directory / 'sub-002_electrodes.tsv'

data = np.loadtxt(electr_file, dtype=str, delimiter='\t', comments=None, encoding='utf-8')
column_names = data[0, :]
info = data[1:, :]

electrode_tsv = OrderedDict()
for i, name in enumerate(column_names):
    electrode_tsv[name] = info[:, i].tolist()

# Load in channel names
ch_names = electrode_tsv['name']

# Load in the xyz coordinates as a float
elec = np.empty(shape=(len(ch_names), 3))
for ind, axis in enumerate(['x', 'y', 'z']):
    elec[:, ind] = list(map(float, electrode_tsv[axis]))

# Create mne montage
montage = mne.channels.make_dig_montage(ch_pos=dict(zip(ch_names, elec)), coord_frame='mni_tal')
print('Created %s channel positions' % len(ch_names))
print(dict(zip(ch_names, elec)))

# Set montage. Warning is issued if channels don't match. Consider getting locations for missing channels.
raw.set_montage(montage, on_missing='warn')

In [ ]:
# Write out files in BIDS format
# Might issue SameFileError if no changes are made to raw. Can be ignored, since _ieeg files don't need to be overwritten.

# Workaround, if raw data has been loaded.
if raw.preload == True:
    fname_fif = bids_path.directory / (bids_path.basename + 'raw.fif')
    raw.save(fname_fif, proj=True, overwrite=True)
    raw = mne.io.read_raw_fif(fname_fif, preload=False, verbose=False)

write_raw_bids(raw, bids_path, overwrite=True, verbose=True)